Background:
- We have a big truck with a specific size
- We have a list of products with different sizes and prices which should be inserted into the truck 
- How to get the maximum utilization and profit of the truck ?  

### Products

In [239]:
# initiating a class for all products
class Product():
    def __init__(self, name, space, price):
        self.name = name
        self.space = space
        self.price = price

In [240]:
# creating products
products_list = []
products_list.append(Product('Refrigerator A', 0.751, 999.90))
products_list.append(Product('Cell phone', 0.00000899, 2199.12))
products_list.append(Product('TV 55', 0.400, 4346.99))
products_list.append(Product("TV 50' ", 0.290, 3999.90))
products_list.append(Product("TV 42' ", 0.200, 2999.00))
products_list.append(Product("Notebook A", 0.00350, 2499.90))
products_list.append(Product("Ventilator", 0.496, 199.90))
products_list.append(Product("Microwave A", 0.0424, 308.66))
products_list.append(Product("Microwave B", 0.0544, 429.90))
products_list.append(Product("Microwave C", 0.0319, 299.29))
products_list.append(Product("Refrigerator B", 0.635, 849.00))
products_list.append(Product("Refrigerator C", 0.870, 1199.89))
products_list.append(Product("Notebook B", 0.498, 1999.90))
products_list.append(Product("Notebook C", 0.527, 3999.00)) 

spaces = []
prices = []
names = []

for product in products_list:
    spaces.append(product.space)
    prices.append(product.price)
    names.append(product.name)
limit = 3
population_size = 20
mutation_probability = 0.01
number_of_generations = 100


In [241]:
for product in products_list:
    print(product.name, '-', product.space, '-', product.price)

Refrigerator A - 0.751 - 999.9
Cell phone - 8.99e-06 - 2199.12
TV 55 - 0.4 - 4346.99
TV 50'  - 0.29 - 3999.9
TV 42'  - 0.2 - 2999.0
Notebook A - 0.0035 - 2499.9
Ventilator - 0.496 - 199.9
Microwave A - 0.0424 - 308.66
Microwave B - 0.0544 - 429.9
Microwave C - 0.0319 - 299.29
Refrigerator B - 0.635 - 849.0
Refrigerator C - 0.87 - 1199.89
Notebook B - 0.498 - 1999.9
Notebook C - 0.527 - 3999.0


### Individual

In [242]:
# Indivdual
# Each indivdual represents the solution
# A set of indivdual forms a population

from random import random

class Individual():
    def __init__(self, spaces, prices, space_limit, generation = 0):
        self.spaces = spaces
        self.prices = prices
        self.space_limit = space_limit
        self.score_evaluation = 0
        self.used_space = 0
        self.generation = generation
        self.chromosome = [] # the actual solution is chromosome

        # Randomly initializing the  in the first step
        for i in range(len(spaces)):
            if random() < 0.5:
                self.chromosome.append('0')
            else:
                self.chromosome.append('1')
            
    def fitness(self):
        # this function returns the total space and total price of the selected products (chromosomes)
        score = 0
        sum_spaces = 0
        for i in range(len(self.chromosome)):
            if self.chromosome[i] == '1':
                score += self.prices[i]
                sum_spaces += self.spaces[i]
        
        # if the space of selected products are more than space available in truck 
        if sum_spaces > self.space_limit:
            # discarding this solution
            score = 1
        self.score_evaluation = score
        self.used_space = sum_spaces

    def crossover(self, other_individual):
        # Random point to cut and crossover
        cut_off = round(random() * len(self.chromosome)) # getting a random number from 0 to the length of chromosome.

        self.child1 = other_individual.chromosome[0:cut_off] + self.chromosome[cut_off::]
        self.child2 = self.chromosome[0:cut_off] + other_individual.chromosome[cut_off::]

        # print(cut_off)
        # print(self.child1)
        # print(self.child2)

        children = [Individual(self.spaces, self.prices, self.space_limit, self.generation + 1),
                    Individual(self.spaces, self.prices, self.space_limit, self.generation + 1)] # generation + 1 means next generation
        children[0].chromosome = self.child1
        children[1].chromosome = self.child2
        return children

    def mutation(self, rate):
        # print('before mutation - ', self.chromosome)
        for i in range(len(self.chromosome)):
            if random() < rate:
                if self.chromosome[i] == '1':
                    self.chromosome[i] = '0'
                else: 
                    self.chromosome[i] = '1'
        # print('after mutation - ', self.chromosome)
        return self

### Genetic Algorithm Class

In [252]:
class GeneticAlgorithm():
    def __init__(self, population_size):
        self.population_size = population_size
        self.population = []
        self.generation = 0
        self.best_solution = None
        self.list_of_solutions = []

    def initialize_population(self, spaces, prices, space_limit):
        # for initializing the population
        for i in range(self.population_size):
            self.population.append(Individual(spaces = spaces, prices = prices, space_limit = space_limit))
        # Initializing the first position as the best solution (just for initializing)
        self.best_solution = self.population[0]
    
    def order_population(self):
        # this function is for ordering the list (sorting)
        # in the first position of the list, we can see the best individual
        self.population = sorted(self.population, key = lambda population: population.score_evaluation, reverse = True)

    def best_individual(self, individual):
        if individual.score_evaluation > self.best_solution.score_evaluation:
            self.best_solution = individual

    def sum_evaluations(self):
        # sum of all the scores for all the individuals
        sum = 0
        for individial in self.population:
            sum += individial.score_evaluation
        return sum

    def select_parent(self, sum_evaluation):
        # Roulette method
        parent = -1 # it will pick one from the population. It's -1 because it won't pick anything from population at first
        random_value = random() * sum_evaluation
        sum = 0
        i = 0
        # print('selecting parents')
        while i < len(self.population) and sum < random_value:
            sum += self.population[i].score_evaluation
            # print(parent)
            parent += 1 # the index of individual 
            i += 1
        return parent

    def visualize_generation(self):
        best = self.population[0]
        print('Generation: ', self.population[0].generation,
        'Total Price: ', best.score_evaluation, 'Space: ', best.used_space, 'Chromosome: ', best.chromosome)

    def solve(self, mutation_probability, number_of_generations, spaces, prices, limit):
        # creating initial population
        self.initialize_population(spaces, prices, limit)
        # evaluate population
        for individual in self.population:
            # print('1st', individual)
            individual.fitness()
        self.order_population()
        self.visualize_generation()
        # stopping criteria
        for generation in range(number_of_generations):
            # select parents
            sum = self.sum_evaluations()
            new_population = []
            for new_individuals in range(0, self.population_size, 2):
                parent1 = self.select_parent(sum)
                parent2 = self.select_parent(sum)
                # cross over
                children = self.population[parent1].crossover(self.population[parent2])
                # mutation
                new_population.append(children[0].mutation(mutation_probability))
                new_population.append(children[1].mutation(mutation_probability))
            # discarding old population
            self.population = list(new_population)
            # evaluate the population
            for individual in self.population:
                # print(individual)
                individual.fitness()
            self.visualize_generation()
            best = self.population[0]
            self.best_individual(best)
        print('*** BEST SOLUTION ***')
        print('Generation: ', self.best_solution.generation,
        'Total Price: ', self.best_solution.score_evaluation, 'Space: ', self.best_solution.used_space, 'Chromosome: ', self.best_solution.chromosome)

        return self.best_solution.chromosome


In [266]:
ga = GeneticAlgorithm(population_size)
result = ga.solve(mutation_probability, number_of_generations, spaces, prices, limit)
print(result)
for i in range(len(products_list)):
    if result[i] == '1':
        print('Name: ', products_list[i].name, '- Price: ', products_list[i].price)


Generation:  0 Total Price:  17282.55 Space:  2.85120899 Chromosome:  ['1', '1', '1', '0', '1', '1', '0', '1', '1', '1', '0', '1', '1', '0']
Generation:  1 Total Price:  10347.8 Space:  1.1285 Chromosome:  ['0', '0', '0', '1', '1', '1', '0', '0', '0', '0', '1', '0', '0', '0']
Generation:  2 Total Price:  14931.86 Space:  1.69420899 Chromosome:  ['0', '1', '1', '0', '0', '1', '0', '1', '1', '1', '1', '0', '0', '1']
Generation:  3 Total Price:  1 Space:  3.0203 Chromosome:  ['1', '0', '0', '1', '0', '0', '1', '0', '1', '1', '0', '1', '0', '1']
Generation:  4 Total Price:  14544.19 Space:  1.8324089900000002 Chromosome:  ['0', '1', '1', '0', '0', '1', '0', '0', '0', '1', '0', '1', '0', '1']
Generation:  5 Total Price:  14344.2 Space:  1.71340899 Chromosome:  ['1', '1', '1', '0', '0', '1', '0', '0', '0', '1', '0', '0', '0', '1']
Generation:  6 Total Price:  16505.56 Space:  2.6868089900000003 Chromosome:  ['1', '1', '0', '1', '1', '1', '0', '1', '0', '1', '0', '1', '1', '0']
Generation:  7